In [1]:
import pandas as pd
import polars as pl
import numpy as np
import datetime as dt
import asyncio
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mplfinance as mpf
from plotly.subplots import make_subplots
from dash import Dash, dcc, html
from typing import Literal
import pandas_market_calendars as mcal
nse = mcal.get_calendar('NSE')

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = '{:.4f}'.format

In [3]:
import sys
sys.path.append('..')
from tooling.filter import find_atm, option_tool
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm
from tooling.enums import Index, AssetClass, StrikeSpread, Spot

In [4]:
async def get_expiry(f_today):
    
    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24 :
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(start_date=nearest_wednesday, end_date=nearest_wednesday).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_nifty(f_today):
    
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

In [128]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
bnf_1min = pd.read_csv('../data/bnf_min.csv')
# bnf_1min = pd.read_csv('../data/nifty_min.csv')
bnf_1min['datetime'] = pd.to_datetime(bnf_1min['datetime'])
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[(bnf_1min['datetime'].dt.year >= 2023) & (bnf_1min['datetime'].dt.year <= 2023)]

In [123]:
# # If Stocks Data ...
# bnf_pandas['datetime'] = pd.to_datetime(bnf_pandas['time'])
# bnf_pandas['datetime'] = bnf_pandas['datetime'].dt.tz_localize(None)
# bnf_pandas = bnf_pandas[bnf_pandas['datetime'].dt.year >= 2017]
# bnf_pandas.drop(columns=['time'], inplace=True)
# # bnf_pandas

In [124]:
# bnf = pl.DataFrame(bnf_pandas)
# bnf = bnf.with_columns([pl.col('datetime').alias('index')]).drop('datetime')
# bnf = bnf.with_columns(pl.col("index").alias("datetime"))
# # bnf

In [125]:
# bnf_1min

In [126]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label='left',
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )

# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [155]:
def calculate_weekly_ranges(df):
    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)
    # Resample to weekly data to get the high and low for each week
    weekly_ranges = df.resample('W-FRI').agg({'high': 'max', 'low': 'min'})
    return weekly_ranges

async def find_breakouts(df, weekly_ranges):
    
    results = []
    multiplier_to_range = 0.005
    expected_mkt_close = 0.03

    portfolio_value = 10_00_000
    index_leverage = 6

    index_name = 'BANKNIFTY'
    index_str_for_opt = 'bnf'
    
    weekly_ranges['datetime'] = pd.to_datetime(weekly_ranges['datetime'])
    weekly_ranges = weekly_ranges[:-1]
    # print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]

    
    for i in range(1, len(weekly_ranges)):
        previous_week = weekly_ranges.iloc[i-1]
        current_week = weekly_ranges.iloc[i]
        print('Current Week : ', current_week['datetime'].date())
        if current_week['datetime'].date() in dates_to_avoid:
            print('Date Avoided')
            continue
        previous_week_high = weekly_ranges.iloc[i-1]['high']
        previous_week_low = weekly_ranges.iloc[i-1]['low']
        previous_week_close = weekly_ranges.iloc[i-1]['close']

        current_week_open = weekly_ranges.iloc[i]['open']
        
        # Filter the 1-minute data for the current week
        # current_week_data = df[(df.index > weekly_ranges.index[i-1]) & (df.index <= current_week)]
        current_week_data = df.loc[(df['datetime'] >= current_week['datetime']) & (df['datetime'] <= (current_week['datetime'] + pd.Timedelta(days=6)))]
        # print("Current Week Data:\n", current_week_data)

        weekly_range = previous_week_high - previous_week_low
        addition_range = multiplier_to_range * weekly_range

        # Check for breakouts
        high_level = current_week_data['open'].iloc[0] + addition_range
        low_level = current_week_data['open'].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)
        
        breakout_high = current_week_data[current_week_data['high'] >= high_level]
        breakout_low = current_week_data[current_week_data['low'] <= low_level]

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]['datetime']
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(high_level * (1+expected_mkt_close) / 100) * 100)
            buy_strike = atm_strike
            sell_strike = atm_strike+100
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry(current_week['datetime'].date() + dt.timedelta(days=5))
            dte = (expiry - breakout_high_time.date()).days
            buy_contract = await get_option_contract_name(
                symbol=index_name,
                strike=buy_strike,
                expiry=expiry,
                opt_type='CE',
            )
            sell_contract = await get_option_contract_name(
                symbol=index_name,
                strike=sell_strike,
                expiry=expiry,
                opt_type='CE',
            ) 
            pe_df_buy = await fetch_option_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=expiry,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=buy_strike,
                asset_class='C',
            )
            pe_df_sell = await fetch_option_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=expiry,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=sell_strike,
                asset_class='C',
            )
            if (not isinstance(pe_df_buy, str)) and (not isinstance(pe_df_sell, str)):
                pe_df_buy = pe_df_buy.to_pandas()
                pe_df_sell = pe_df_sell.to_pandas()
                entry_price_buy = pe_df_buy.iloc[0]['c']
                entry_price_sell = pe_df_sell.iloc[0]['c']

            else:
                entry_price_buy = float('nan')
                entry_price_sell = float('nan')
            
            # Exit Scenario
            current_week_data_after_entry = current_week_data[(current_week_data['datetime'] > breakout_high_time) & (current_week_data['datetime'].dt.date <= expiry - dt.timedelta(days=0))]
            # print(current_week_data_after_entry)
            low_breach = current_week_data_after_entry[current_week_data_after_entry['low'] < previous_week_low]
            # print(low_breach)
            # if len(low_breach) != 0:
            #     exit_time = low_breach.iloc[0]['datetime']
            #     # print(exit_time)
            #     remark = 'SL Hit'
            #     # print(remark, exit_time)
            #     # print(pe_df)
            #     if not ((math.isnan(entry_price_buy)) and (math.isnan(entry_price_sell))):
            #         # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
            #         exit_price_buy = pe_df_buy.loc[(pe_df_buy['datetime'].dt.date >= exit_time.date()) & (pe_df_buy['datetime'].dt.hour >= exit_time.hour) & (pe_df_buy['datetime'].dt.minute >= exit_time.minute), 'c'].iloc[0]
            #         exit_price_sell = pe_df_sell.loc[(pe_df_sell['datetime'].dt.date >= exit_time.date()) & (pe_df_sell['datetime'].dt.hour >= exit_time.hour) & (pe_df_sell['datetime'].dt.minute >= exit_time.minute), 'c'].iloc[0]
                    
            #     else:
            #         exit_price_buy = float('nan')
            #         exit_price_sell = float('nan')
            #         # print(exit_price)
            # else:
            exit_time = dt.datetime.combine(expiry - dt.timedelta(days=0), dt.time(15, 30))
            remark = 'Expiry Closing'
            # print(remark, exit_time)
            if not ((math.isnan(entry_price_buy)) and (math.isnan(entry_price_sell))):
                exit_price_buy = pe_df_buy.iloc[-1]['c']
                exit_price_sell = pe_df_sell.iloc[-1]['c']
            else:
                exit_price_buy = float('nan')
                exit_price_sell = float('nan')
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = (portfolio_value * index_leverage / sell_strike)
            # slippage = 0.01 * (entry_price_buy + exit_price_sell + exit_price_buy + exit_price_sell)
            slippage = 0
            max_loss_points = abs(entry_price_buy - entry_price_sell)
            max_profit = (abs(buy_strike - sell_strike) - abs(entry_price_buy - entry_price_sell)) * qty
            max_loss = abs(entry_price_buy - entry_price_sell) * qty
            points = (exit_price_buy - entry_price_buy) + (entry_price_sell - exit_price_sell)
            pnl = qty * points
            final_pnl = qty * (points - slippage)
            unit = {
                'Week': previous_week['datetime'].date(),
                'Week High': previous_week_high,
                'Week Low': previous_week_low,
                'Weekly Range': weekly_range,
                'Current Week Open': current_week_open,
                'Multiplier to Range': f'{int(multiplier_to_range * 100)}%',
                'Entry Level Long': high_level,
                'Entry Level Short': low_level,
                'Break Type': 'high', 
                'Buy Strike': buy_strike,
                'Sell Strike': sell_strike,
                'Expiry': expiry,
                'DTE': dte,
                'Buy Contract': buy_contract,
                'Sell Contract': sell_contract,
                'Option Types': buy_contract[-2:],
                'Entry Time': breakout_high_time,
                'Buy Strike Entry Price': entry_price_buy,
                'Sell Strike Entry Price': entry_price_sell,
                'Exit Time': exit_time,
                'Buy Strike Exit Price': exit_price_buy,
                'Sell Strike Exit Price': exit_price_sell,
                'Debit': max_loss_points,
                'Max Profit': max_profit,
                'Max Loss': max_loss,
                'Remark': remark,
                'Points': points,
                'Slippage': slippage,
                'Points w cs': points - slippage,
                'Qty': qty,
                'PnL': pnl,
                'PnL w cs': final_pnl,
                'ROI%': (pnl * 100 / portfolio_value),
                'ROI% w cs': (final_pnl * 100 / portfolio_value)
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]['datetime']
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(low_level * (1-expected_mkt_close) / 100) * 100)
            buy_strike = atm_strike
            sell_strike = atm_strike-100
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry(current_week['datetime'].date() + dt.timedelta(days=5))
            dte = (expiry - breakout_low_time.date()).days
            buy_contract = await get_option_contract_name(
                symbol=index_name,
                strike=buy_strike,
                expiry=expiry,
                opt_type='PE',
            )
            sell_contract = await get_option_contract_name(
                symbol=index_name,
                strike=sell_strike,
                expiry=expiry,
                opt_type='PE',
            )
            ce_df_buy = await fetch_option_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=expiry,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=buy_strike,
                asset_class='P',
            )
            ce_df_sell = await fetch_option_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=expiry,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=sell_strike,
                asset_class='P',
            )
            # print(len(ce_df))
            if (not isinstance(ce_df_buy, str)) and (not isinstance(ce_df_sell, str)):
                ce_df_buy = ce_df_buy.to_pandas()
                ce_df_sell = ce_df_sell.to_pandas()
                entry_price_buy = ce_df_buy.iloc[0]['c']
                entry_price_sell = ce_df_sell.iloc[0]['c']
            
            else:
                entry_price_buy = float('nan')
                entry_price_sell = float('nan')
                
            # Exit Scenario
            current_week_data_after_entry = current_week_data[(current_week_data['datetime'] > breakout_low_time) & (current_week_data['datetime'].dt.date <= expiry - dt.timedelta(days=0))]
            # print(current_week_data_after_entry)
            high_breach = current_week_data_after_entry[current_week_data_after_entry['high'] > previous_week_high]
            # print(high_breach)
            # if len(high_breach) != 0:
            #     exit_time = high_breach.iloc[0]['datetime']
            #     # print(exit_time)
            #     remark = 'SL Hit'
            #     # print(remark, exit_time)
            #     if not ((math.isnan(entry_price_buy)) and (math.isnan(entry_price_sell))):
            #         # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
            #         exit_price_buy = ce_df_buy.loc[(ce_df_buy['datetime'].dt.date >= exit_time.date()) & (ce_df_buy['datetime'].dt.hour >= exit_time.hour) & (ce_df_buy['datetime'].dt.minute >= exit_time.minute), 'c'].iloc[0]
            #         exit_price_sell = ce_df_sell.loc[(ce_df_sell['datetime'].dt.date >= exit_time.date()) & (ce_df_sell['datetime'].dt.hour >= exit_time.hour) & (ce_df_sell['datetime'].dt.minute >= exit_time.minute), 'c'].iloc[0]
            #     else:
            #         entry_price_buy = float('nan')
            #         entry_price_sell = float('nan')
        else:
            exit_time = dt.datetime.combine(expiry - dt.timedelta(days=0), dt.time(15, 30))
            remark = 'Expiry Closing'
            # print(remark, exit_time)
            if not ((math.isnan(entry_price_buy)) and (math.isnan(entry_price_sell))):
                exit_price_buy = ce_df_buy.iloc[-1]['c']
                exit_price_sell = ce_df_sell.iloc[-1]['c']
            else:
                exit_price_buy = float('nan')
                exit_price_sell = float('nan')
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            
            qty = (portfolio_value * index_leverage / sell_strike)
            # slippage = 0.01 * (entry_price_buy + exit_price_sell + exit_price_buy + exit_price_sell)
            slippage = 0
            max_loss_points = abs(entry_price_buy - entry_price_sell)
            max_profit = (abs(buy_strike - sell_strike) - abs(entry_price_buy - entry_price_sell)) * qty
            max_loss = abs(entry_price_buy - entry_price_sell) * qty
            points = (exit_price_buy - entry_price_buy) + (entry_price_sell - exit_price_sell)
            pnl = qty * points
            final_pnl = qty * (points - slippage)
            unit = {
                'Week': previous_week['datetime'].date(),
                'Week High': previous_week_high,
                'Week Low': previous_week_low,
                'Weekly Range': weekly_range,
                'Current Week Open': current_week_open,
                'Multiplier to Range': f'{int(multiplier_to_range * 100)}%',
                'Entry Level Long': high_level,
                'Entry Level Short': low_level,
                'Break Type': 'low',
                'Buy Strike': buy_strike,
                'Sell Strike': sell_strike,
                'Expiry': expiry,
                'DTE': dte,
                'Buy Contract': buy_contract,
                'Sell Contract': sell_contract,
                'Option Types': buy_contract[-2:],
                'Entry Time': breakout_low_time,
                'Buy Strike Entry Price': entry_price_buy,
                'Sell Strike Entry Price': entry_price_sell,
                'Exit Time': exit_time,
                'Buy Strike Exit Price': exit_price_buy,
                'Sell Strike Exit Price': exit_price_sell,
                'Debit': max_loss_points,
                'Max Profit': max_profit,
                'Max Loss': max_loss,
                'Remark': remark,
                'Points': points,
                'Slippage': slippage,
                'Points w cs': points - slippage,
                'Qty': qty,
                'PnL': pnl,
                'PnL w cs': final_pnl,
                'ROI%': (pnl * 100 / portfolio_value),
                'ROI% w cs': (final_pnl * 100 / portfolio_value)
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results

async def trade():
    df = bnf_1min
    # weekly_ranges = calculate_weekly_ranges(df)
    weekly_ranges = resample(pl.DataFrame(df), '7d', pd.Timedelta(days=1))
    weekly_ranges = weekly_ranges.to_pandas()
    
    breakouts = await find_breakouts(df, weekly_ranges)
    breakouts_pandas = pd.DataFrame(breakouts)
    # breakouts_polars = pl.DataFrame(breakouts)
    return breakouts_pandas

tradebook = await trade()
# tradebook

Current Week :  2023-01-06
Current Week :  2023-01-13
Current Week :  2023-01-20
Current Week :  2023-01-27
Current Week :  2023-02-03
Current Week :  2023-02-10
Current Week :  2023-02-17
Current Week :  2023-02-24
Current Week :  2023-03-03
Current Week :  2023-03-10
Current Week :  2023-03-17
Current Week :  2023-03-24
Current Week :  2023-03-31
Current Week :  2023-04-07
Current Week :  2023-04-14
Current Week :  2023-04-21
Current Week :  2023-04-28
Current Week :  2023-05-05
Current Week :  2023-05-12
Current Week :  2023-05-19
Current Week :  2023-05-26
Current Week :  2023-06-02
Current Week :  2023-06-09
Current Week :  2023-06-16
Current Week :  2023-06-23
Current Week :  2023-06-30
Current Week :  2023-07-07
Current Week :  2023-07-14
Current Week :  2023-07-21
Current Week :  2023-07-28
Current Week :  2023-08-04
Current Week :  2023-08-11
Current Week :  2023-08-18
Current Week :  2023-08-25
Current Week :  2023-09-01
Current Week :  2023-09-08
Current Week :  2023-09-15
C

In [156]:
tradebook

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Buy Strike,Sell Strike,Expiry,DTE,Buy Contract,Sell Contract,Option Types,Entry Time,Buy Strike Entry Price,Sell Strike Entry Price,Exit Time,Buy Strike Exit Price,Sell Strike Exit Price,Debit,Max Profit,Max Loss,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs
0,2022-12-30,43577.3500,42300.6000,1276.7500,42649.7000,0%,42656.0837,42643.3162,high,43900,44000,2023-01-12,6,BANKNIFTY2311243900CE,BANKNIFTY2311244000CE,CE,2023-01-06 09:15:00,34.1000,27.8000,2023-01-12 15:30:00,0.0500,0.0500,6.3000,12777.2727,859.0909,Expiry Closing,-6.3000,0,-6.3000,136.3636,-859.0909,-859.0909,-0.0859,-0.0859
1,2023-01-06,42711.1500,41731.0000,980.1500,42171.2500,0%,42176.1508,42166.3492,high,43400,43500,2023-01-19,6,BANKNIFTY2311943400CE,BANKNIFTY2311943500CE,CE,2023-01-13 09:35:00,38.2000,31.2500,2023-01-19 15:30:00,0.1500,0.1000,6.9500,12834.4828,958.6207,Expiry Closing,-6.9000,0,-6.9000,137.9310,-951.7241,-951.7241,-0.0952,-0.0952
2,2023-01-13,42712.3000,41863.0000,849.3000,42516.0500,0%,42520.2965,42511.8035,high,43800,43900,2023-01-25,5,BANKNIFTY23JAN43800CE,BANKNIFTY23JAN43900CE,CE,2023-01-20 09:15:00,11.8500,10.4000,2023-01-25 15:30:00,0.1000,0.1000,1.4500,13469.2483,198.1777,Expiry Closing,-1.4500,0,-1.4500,136.6743,-198.1777,-198.1777,-0.0198,-0.0198
3,2023-01-20,43077.0000,41541.2500,1535.7500,41382.3500,0%,41390.0287,41374.6712,high,42600,42700,2023-02-02,6,BANKNIFTY2320242600CE,BANKNIFTY2320242700CE,CE,2023-01-27 09:15:00,118.7500,104.3000,2023-02-02 15:30:00,0.1000,0.0500,14.4500,12021.0773,2030.4450,Expiry Closing,-14.4000,0,-14.4000,140.5152,-2023.4192,-2023.4192,-0.2023,-0.2023
4,2023-01-27,42011.2000,39421.1000,2590.1000,41019.3500,0%,41032.3005,41006.3995,high,42300,42400,2023-02-09,6,BANKNIFTY2320942300CE,BANKNIFTY2320942400CE,CE,2023-02-03 09:15:00,86.6000,73.9000,2023-02-09 15:30:00,0.1000,0.0500,12.7000,12353.7736,1797.1698,Expiry Closing,-12.6500,0,-12.6500,141.5094,-1790.0943,-1790.0943,-0.1790,-0.1790
5,2023-02-03,41789.2500,40615.9000,1173.3500,41452.4000,0%,41458.2668,41446.5333,high,42700,42800,2023-02-16,6,BANKNIFTY2321642700CE,BANKNIFTY2321642800CE,CE,2023-02-10 09:15:00,28.8500,23.6500,2023-02-16 15:30:00,0.1000,0.0500,5.2000,13289.7196,728.9720,Expiry Closing,-5.1500,0,-5.1500,140.1869,-721.9626,-721.9626,-0.0722,-0.0722
6,2023-02-17,41514.2000,39603.8000,1910.4000,40259.1000,0%,40268.6520,40249.5480,high,41500,41600,2023-03-02,6,BANKNIFTY2330241500CE,BANKNIFTY2330241600CE,CE,2023-02-24 09:23:00,30.4000,24.9000,2023-03-02 15:30:00,0.0500,0.0500,5.5000,13629.8077,793.2692,Expiry Closing,-5.5000,0,-5.5000,144.2308,-793.2692,-793.2692,-0.0793,-0.0793
7,2023-02-24,40771.3000,39750.0000,1021.3000,40671.4000,0%,40676.5065,40666.2935,high,41900,42000,2023-03-09,6,BANKNIFTY2330941900CE,BANKNIFTY2330942000CE,CE,2023-03-03 09:15:00,14.3000,12.4000,2023-03-09 15:30:00,0.1000,0.1000,1.9000,14014.2857,271.4286,Expiry Closing,-1.9000,0,-1.9000,142.8571,-271.4286,-271.4286,-0.0271,-0.0271
8,2023-03-03,41667.8000,40630.7000,1037.1000,40805.2500,0%,40810.4355,40800.0645,high,42000,42100,2023-03-16,6,BANKNIFTY2331642000CE,BANKNIFTY2331642100CE,CE,2023-03-10 09:15:00,26.6000,21.9500,2023-03-16 15:30:00,0.1000,0.2000,4.6500,13589.0736,662.7078,Expiry Closing,-4.7500,0,-4.7500,142.5178,-676.9596,-676.9596,-0.0677,-0.0677
9,2023-03-10,40811.2000,38617.8000,2193.4000,39442.4000,0%,39453.3670,39431.4330,high,40600,40700,2023-03-23,6,BANKNIFTY2332340600CE,BANKNIFTY2332340700CE,CE,2023-03-17 09:15:00,78.2000,64.6500,2023-03-23 15:30:00,0.1000,0.1500,13.5500,12744.4717,1997.5430,Expiry Closing,-13.6000,0,-13.6000,147.4201,-2004.9140,-2004.9140,-0.2005,-0.2005


In [157]:
# positive = tradebook[tradebook['Points']<0]
tradebook['Points'].sum()
# tradebook['ROI%'].sum()
# no_value = tradebook[(tradebook['ROI%'] > 0) | (tradebook['ROI%'] < 0)]
# len(no_value)

-95.55

In [158]:
# x = tradebook['Entry Time'].iloc[0].year
tradebook['Entry Time'] = pd.to_datetime(tradebook['Entry Time'])
tradebook['Trade Year'] = tradebook['Entry Time'].dt.year

In [159]:
stats_df8 = pd.DataFrame(index=range(2017, 2025), columns=['Total ROI', 'Total Trades', 'Win Rate', 'Avg Profit% per Trade', 'Avg Loss% per Trade', 'Max Drawdown', 'ROI/DD Ratio'])
combined_df_sorted = tradebook
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted['Trade Year'] == year)]
    
    # Calculate total ROI
    total_roi = year_trades['ROI% w cs'].sum()
    
    # Calculate total number of trades
    total_trades = len(year_trades)
    
    # Calculate win rate
    win_rate = (year_trades['ROI% w cs'] > 0).mean()*100
    
    # Calculate average profit per trade
    avg_profit = year_trades[year_trades['ROI% w cs'] > 0]['ROI% w cs'].mean()
    
    # Calculate average loss per trade
    avg_loss = year_trades[year_trades['ROI% w cs'] < 0]['ROI% w cs'].mean()
    
    # Calculate maximum drawdown
    max_drawdown = (year_trades['ROI% w cs'].cumsum() - year_trades['ROI% w cs'].cumsum().cummax()).min()
    
    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)
    
    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [total_roi, total_trades, win_rate, avg_profit, avg_loss, max_drawdown, roi_dd_ratio]

# Calculate overall statistics
overall_total_roi = stats_df8['Total ROI'].sum()
overall_total_trades = stats_df8['Total Trades'].sum()
overall_win_rate = (combined_df_sorted['ROI% w cs'] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted['ROI% w cs'] > 0]['ROI% w cs'].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted['ROI% w cs'] < 0]['ROI% w cs'].mean()
overall_max_drawdown = (combined_df_sorted['ROI% w cs'].cumsum() - combined_df_sorted['ROI% w cs'].cumsum().cummax()).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc['Overall'] = [overall_total_roi, overall_total_trades, overall_win_rate, overall_avg_profit, overall_avg_loss, overall_max_drawdown, overall_roi_dd_ratio]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,0,NaN,NaN,NaN,NaN,NaN
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN
2022,0.0000,0,NaN,NaN,NaN,NaN,NaN
2023,-1.3513,49,2.0408,1.0598,-0.0513,-2.1086,-0.6409
2024,0.0000,0,NaN,NaN,NaN,NaN,NaN
Overall,-1.3513,49.0000,2.0408,1.0598,-0.0513,-2.1086,-0.6409


50% MOVE VARIATION FROM CURRENT OPEN, ENTRY AT OPP LEVELS, SL AT PRV. WK H/L

In [ ]:
stats_of_trades = tradebook
stats_of_trades['Cumulative ROI%'] = stats_of_trades.groupby('Option Type')['ROI%'].cumsum()
stats_of_trades['Running Max ROI%'] = stats_of_trades.groupby('Option Type')['Cumulative ROI%'].cummax()
stats_of_trades['Drawdown'] = stats_of_trades['Cumulative ROI%'] - stats_of_trades['Running Max ROI%']
max_dd_distribution = stats_of_trades.groupby('Option Type')['Drawdown'].min().reset_index()
max_dd_distribution.rename(columns={'Drawdown': 'Max Drawdown'}, inplace=True)
roi_distribution = stats_of_trades.groupby('Option Type')['ROI%'].sum().reset_index()
grouped = pd.merge(roi_distribution, max_dd_distribution, on='Option Type')
grouped['ROI/DD Ratio'] = grouped['ROI%'] / grouped['Max Drawdown'].abs()
grouped

In [136]:
tradebook.to_csv('DS v1 JJMS.csv')

In [47]:
# def round_to_nearest_100(value):
#     return 100 * round(value / 100)
    
# async def add_hedges(df):
#     # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
#     hedge_pct = 1
#     index_name = 'BANKNIFTY'
#     index_str_for_opt = 'bnf'
    
#     # df['Hedge Strike'] = df.apply(lambda row: row['Strike'] + round_to_nearest_100((hedge_pct / 100) * row['Strike']) if 'C' in row['Contract'] else row['Strike'] - round_to_nearest_100((hedge_pct / 100) * row['Strike']), axis=1)
#     df['Hedge Strike'] = df.apply(lambda row: round_to_nearest_100(row['Week High']) if row['Option Type'] == 'CE' else round_to_nearest_100(row['Week Low']), axis=1)
    
#     df['Hedge Contract'] = df.apply(
#         lambda row: get_option_contract_name2(
#             index_name, 
#             row['Hedge Strike'], 
#             row['Expiry'], 
#             row['Option Type']
#         ), 
#         axis=1
#     )

#     df['Hedge Entry Price'] = np.nan
#     df['Hedge Exit Price'] = np.nan

#     for i in range(0, len(df)):
#         print(df.iloc[i]['Week'])
#         hedge_strike = df.iloc[i]['Hedge Strike']
#         hedge_expiry = df.iloc[i]['Expiry']
#         hedge_opt_type = df.iloc[i]['Option Type'][-2:-1]
#         hedge_entry_datetime = df.iloc[i]['Entry Time']
#         hedge_exit_datetime = df.iloc[i]['Exit Time']

#         hedge_df = await fetch_option_data(
#             index=index_str_for_opt,
#             start_date=hedge_entry_datetime.date(),
#             end_date=hedge_exit_datetime.date(),
#             start_time=hedge_entry_datetime.time(),
#             end_time=hedge_exit_datetime.time(),
#             expiry=hedge_expiry,
#             strike=hedge_strike,
#             asset_class=hedge_opt_type,
#         )
#         if not isinstance(hedge_df, str):
#             hedge_df = hedge_df.to_pandas()
#         else:
#             print(hedge_df)
#             continue
#         # print(hedge_df)

#         hedge_entry_price = hedge_df.iloc[0]['c']
#         hedge_exit_price = hedge_df.iloc[-1]['c']

#         print(hedge_entry_price, hedge_exit_price)

#         df.loc[i, 'Hedge Entry Price'] = hedge_entry_price
#         df.loc[i, 'Hedge Exit Price'] = hedge_exit_price
        
    
#     return df

# tb_with_hedge = await add_hedges(tradebook)
# # tb_with_hedge

In [11]:
# tb_with_hedge = pd.read_csv('JJMS TB w Hedges 1%.csv')
# tb_with_hedge

In [46]:
# tb_with_hedge

In [99]:
tb_with_hedge['Hedge Points'] = tb_with_hedge["Hedge Exit Price"] - tb_with_hedge["Hedge Entry Price"]
tb_with_hedge['PnL'] = tb_with_hedge['Points'] * tb_with_hedge['Qty']
tb_with_hedge['Hedge PnL'] = tb_with_hedge['Hedge Points'] * tb_with_hedge['Qty']
tb_with_hedge['Total PnL'] = tb_with_hedge['PnL'] + tb_with_hedge['Hedge PnL']

In [101]:
tb_with_hedge['Total ROI%'] = tb_with_hedge['Total PnL'] * 100 / 1000000
tb_with_hedge['Entry Time'] = pd.to_datetime(tb_with_hedge['Entry Time'])
tb_with_hedge['Trade Year'] = tb_with_hedge['Entry Time'].dt.year

In [102]:
stats_of_trades = tb_with_hedge
stats_of_trades['Cumulative ROI%'] = stats_of_trades.groupby('Option Type')['Total ROI%'].cumsum()
stats_of_trades['Running Max ROI%'] = stats_of_trades.groupby('Option Type')['Cumulative ROI%'].cummax()
stats_of_trades['Drawdown'] = stats_of_trades['Cumulative ROI%'] - stats_of_trades['Running Max ROI%']
max_dd_distribution = stats_of_trades.groupby('Option Type')['Drawdown'].min().reset_index()
max_dd_distribution.rename(columns={'Drawdown': 'Max Drawdown'}, inplace=True)
roi_distribution = stats_of_trades.groupby('Option Type')['Total ROI%'].sum().reset_index()
grouped = pd.merge(roi_distribution, max_dd_distribution, on='Option Type')
grouped['ROI/DD Ratio'] = grouped['Total ROI%'] / grouped['Max Drawdown'].abs()
grouped

,Option Type,Total ROI%,Max Drawdown,ROI/DD Ratio
0,CE,98.4500,-56.7756,1.7340
1,PE,168.3706,-31.7593,5.3015


In [104]:
stats_df8 = pd.DataFrame(index=range(2017, 2025), columns=['Total ROI', 'Total Trades', 'Win Rate', 'Avg Profit% per Trade', 'Avg Loss% per Trade', 'Max Drawdown', 'ROI/DD Ratio'])
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted['Trade Year'] == year)]
    
    # Calculate total ROI
    total_roi = year_trades['Total ROI%'].sum()
    
    # Calculate total number of trades
    total_trades = len(year_trades)
    
    # Calculate win rate
    win_rate = (year_trades['Total ROI%'] > 0).mean()*100
    
    # Calculate average profit per trade
    avg_profit = year_trades[year_trades['Total ROI%'] > 0]['Total ROI%'].mean()
    
    # Calculate average loss per trade
    avg_loss = year_trades[year_trades['Total ROI%'] < 0]['Total ROI%'].mean()
    
    # Calculate maximum drawdown
    max_drawdown = (year_trades['Total ROI%'].cumsum() - year_trades['Total ROI%'].cumsum().cummax()).min()
    
    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)
    
    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [total_roi, total_trades, win_rate, avg_profit, avg_loss, max_drawdown, roi_dd_ratio]

# Calculate overall statistics
overall_total_roi = stats_df8['Total ROI'].sum()
overall_total_trades = stats_df8['Total Trades'].sum()
overall_win_rate = (combined_df_sorted['Total ROI%'] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted['Total ROI%'] > 0]['Total ROI%'].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted['Total ROI%'] < 0]['Total ROI%'].mean()
overall_max_drawdown = (combined_df_sorted['Total ROI%'].cumsum() - combined_df_sorted['Total ROI%'].cumsum().cummax()).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc['Overall'] = [overall_total_roi, overall_total_trades, overall_win_rate, overall_avg_profit, overall_avg_loss, overall_max_drawdown, overall_roi_dd_ratio]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,22.9620,72,63.8889,1.2029,-1.7983,-15.1240,1.5183
2018,39.5779,65,78.4615,1.2398,-2.6279,-9.2966,4.2573
2019,41.1652,71,78.8732,1.2043,-3.2847,-15.6733,2.6265
2020,69.3026,64,79.6875,2.9782,-6.8822,-41.2048,1.6819
2021,32.2425,72,69.4444,1.5637,-2.2970,-27.9458,1.1538
2022,26.5982,74,71.6216,1.7530,-3.3156,-26.6650,0.9975
2023,30.9373,79,65.8228,1.0237,-1.1734,-5.5672,5.5571
2024,4.0348,13,84.6154,0.7280,-3.9737,-3.9737,1.0154
Overall,266.8206,510.0000,72.5490,1.5411,-2.8356,-41.2048,6.4755


In [105]:
a = tb_with_hedge['PnL'].sum()
b = tb_with_hedge['Hedge PnL'].sum()
c = tb_with_hedge['Total PnL'].sum()
d = tb_with_hedge['Total ROI%'].sum()
print(a,b,c,d)

4662425.683406735 -1980922.9054644075 2668205.617525276 266.82056175252757


In [106]:
# tb_with_hedge.to_csv('JJMS TB w Hedges 1%.csv')

In [45]:
# # tb_with_hedge['DD%'] = (tb_with_hedge['Total ROI%'].cumsum() - tb_with_hedge['Total ROI%'].cumsum().cummax())
# tb_with_hedge.drop(columns=['Cumulative ROI%', 'Running Max ROI%', 'Drawdown'], inplace=True)
# tb_with_hedge

In [27]:
tb_with_hedge['ROI%'] = tb_with_hedge['PnL'] * 100 / 1000000

In [113]:
tb_with_hedge['Hedge PnL'].sum()

-1980922.9054644075

In [116]:
stats_df9 = pd.DataFrame(index=range(2017, 2025), columns=['Total ROI', 'Total Trades', 'Win Rate', 'Avg Profit% per Trade', 'Avg Loss% per Trade', 'Max Drawdown', 'ROI/DD Ratio'])
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted['Trade Year'] == year)]
    
    # Calculate total ROI
    total_roi = year_trades['Total ROI%'].sum()
    
    # Calculate total number of trades
    total_trades = len(year_trades)
    
    # Calculate win rate
    win_rate = (year_trades['Total ROI%'] > 0).mean()*100
    
    # Calculate average profit per trade
    avg_profit = year_trades[year_trades['Total ROI%'] > 0]['Total ROI%'].mean()
    
    # Calculate average loss per trade
    avg_loss = year_trades[year_trades['Total ROI%'] < 0]['Total ROI%'].mean()
    
    # Calculate maximum drawdown
    max_drawdown = (year_trades['Total ROI%'].cumsum() - year_trades['Total ROI%'].cumsum().cummax()).min()
    
    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)
    
    # Store the statistics in the DataFrame
    stats_df9.loc[year] = [total_roi, total_trades, win_rate, avg_profit, avg_loss, max_drawdown, roi_dd_ratio]

# Calculate overall statistics
overall_total_roi = stats_df9['Total ROI'].sum()
overall_total_trades = stats_df9['Total Trades'].sum()
overall_win_rate = (combined_df_sorted['ROI% w cs'] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted['Total ROI%'] > 0]['Total ROI%'].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted['Total ROI%'] < 0]['Total ROI%'].mean()
overall_max_drawdown = (combined_df_sorted['Total ROI%'].cumsum() - combined_df_sorted['Total ROI%'].cumsum().cummax()).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df9.loc['Overall'] = [overall_total_roi, overall_total_trades, overall_win_rate, overall_avg_profit, overall_avg_loss, overall_max_drawdown, overall_roi_dd_ratio]
stats_df9

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,22.9620,72,63.8889,1.2029,-1.7983,-15.1240,1.5183
2018,39.5779,65,78.4615,1.2398,-2.6279,-9.2966,4.2573
2019,41.1652,71,78.8732,1.2043,-3.2847,-15.6733,2.6265
2020,69.3026,64,79.6875,2.9782,-6.8822,-41.2048,1.6819
2021,32.2425,72,69.4444,1.5637,-2.2970,-27.9458,1.1538
2022,26.5982,74,71.6216,1.7530,-3.3156,-26.6650,0.9975
2023,30.9373,79,65.8228,1.0237,-1.1734,-5.5672,5.5571
2024,4.0348,13,84.6154,0.7280,-3.9737,-3.9737,1.0154
Overall,266.8206,510.0000,69.8039,1.5411,-2.8356,-41.2048,6.4755


In [42]:
tradebook['Cumulative ROI%'] = tradebook['ROI% w cs'].cumsum()
tradebook['Running Max ROI%'] = tradebook['Cumulative ROI%'].cummax()
tradebook['Drawdown'] = tradebook['Cumulative ROI%'] - tradebook['Running Max ROI%']
tradebook

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Qty,PnL,ROI%,Trade Year,Slippage in pts,PnL w cs,ROI% w cs,Cumulative ROI%,Running Max ROI%,Drawdown
0,2019-12-27,32463.1000,32058.5500,404.5500,32326.9500,50%,32529.2250,32124.6750,low,32500,2020-01-09,6,BANKNIFTY2010932500CE,CE,2020-01-03 11:03:00,99.9500,2020-01-08 15:20:00,0.0500,Expiry Closing,99.9000,199.8203,19962.0495,1.9962,2020,1.0000,19762.2292,1.9762,1.9762,1.9762,0.0000
1,2020-01-10,32345.2000,31671.5000,673.7000,31571.6500,50%,31908.5000,31234.8000,high,31200,2020-01-23,3,BANKNIFTY2012331200PE,PE,2020-01-20 09:15:00,70.4000,2020-01-20 09:21:00,78.6500,SL Hit,-8.2500,203.7075,-1680.5867,-0.1681,2020,1.4905,-1984.2127,-0.1984,1.7778,1.9762,-0.1984
2,2020-01-10,32345.2000,31671.5000,673.7000,31571.6500,50%,31908.5000,31234.8000,low,31900,2020-01-23,3,BANKNIFTY2012331900CE,CE,2020-01-20 13:00:00,63.9000,2020-01-22 15:20:00,0.0500,Expiry Closing,63.8500,203.7075,13006.7223,1.3007,2020,0.6395,12876.4514,1.2876,3.0654,3.0654,0.0000
3,2020-01-24,31374.1500,30522.0500,852.1000,30950.5000,50%,31376.5500,30524.4500,low,31400,2020-02-06,5,BANKNIFTY2020631400CE,CE,2020-02-01 09:15:00,228.9500,2020-02-05 15:20:00,0.1500,Expiry Closing,228.8000,207.1611,47398.4552,4.7398,2020,2.2910,46923.8492,4.6924,7.7578,7.7578,0.0000
4,2020-02-07,31598.5500,30958.3500,640.2000,31281.8500,50%,31601.9500,30961.7500,low,31600,2020-02-20,6,BANKNIFTY2022031600CE,CE,2020-02-14 10:55:00,44.0000,2020-02-19 15:20:00,0.0500,Expiry Closing,43.9500,205.6835,9039.7903,0.9040,2020,0.4405,8949.1867,0.8949,8.6528,8.6528,0.0000
5,2020-02-21,30702.1500,29918.8500,783.3000,29501.5500,50%,29893.2000,29109.9000,low,29900,2020-03-05,6,BANKNIFTY2030529900CE,CE,2020-02-28 15:00:00,129.2000,2020-03-04 15:20:00,0.1000,Expiry Closing,129.1000,217.4408,28071.6016,2.8072,2020,1.2930,27790.4507,2.7790,11.4318,11.4318,0.0000
6,2020-02-28,29778.6500,28125.4500,1653.2000,27649.9500,50%,28476.5500,26823.3500,low,28500,2020-03-12,3,BANKNIFTY2031228500CE,CE,2020-03-09 09:37:00,33.1000,2020-03-11 15:20:00,0.2000,Expiry Closing,32.9000,228.2580,7509.6878,0.7510,2020,0.3330,7433.6779,0.7434,12.1752,12.1752,0.0000
7,2020-03-06,28042.5000,23613.1000,4429.4000,22720.7500,50%,24935.4500,20506.0500,high,20500,2020-03-19,6,BANKNIFTY2031920500PE,PE,2020-03-13 12:37:00,297.0500,2020-03-16 09:24:00,297.0500,SL Hit,0.0000,260.6731,0.0000,0.0000,2020,5.9410,-1548.6586,-0.1549,12.0203,12.1752,-0.1549
8,2020-03-06,28042.5000,23613.1000,4429.4000,22720.7500,50%,24935.4500,20506.0500,low,24900,2020-03-19,1,BANKNIFTY2031924900CE,CE,2020-03-18 12:32:00,9.0000,2020-03-18 15:20:00,0.1000,Expiry Closing,8.9000,260.6731,2319.9902,0.2320,2020,0.0910,2296.2690,0.2296,12.2499,12.2499,0.0000
9,2020-03-13,25654.9500,18680.6500,6974.3000,19898.3000,50%,23385.4500,16411.1500,low,23400,2020-03-26,2,BANKNIFTY20MAR23400CE,CE,2020-03-24 09:52:00,34.0500,2020-03-25 15:20:00,1.0000,Expiry Closing,33.0500,277.9506,9186.2675,0.9186,2020,0.3505,9088.8458,0.9089,13.1588,13.1588,0.0000


In [43]:
tradebook.to_csv('2020 Trades JJMS OS.csv')